# Aqui vamos a hacer un procesamiento basico de datos, y estadisticas relevantes

In [ ]:
# === Requisitos (una sola vez) ===
# pip install ydata-profiling[notebook] scipy

#Se esta usando un ambiente de conda de python 3.12.11 
import os
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport

# ------- Config --------
FOLDER = "Datos"          # Carpeta con CSVs
REPORT = "profile.html"   # Salida del reporte
SAMPLE = None             # e.g., 20000 para muestrear filas si es muy grande

# 1) Cargar y unir CSVs
files = [f for f in os.listdir(FOLDER) if f.lower().endswith(".csv")]
if not files:
    raise FileNotFoundError(f"No hay CSVs en '{FOLDER}'")

dfs = []
for f in files:
    path = os.path.join(FOLDER, f)
    # dtype=object evita inferencias agresivas; luego convertimos
    df_part = pd.read_csv(path, dtype=object, low_memory=False)
    df_part["__archivo_origen"] = f
    dfs.append(df_part)

df = pd.concat(dfs, ignore_index=True)

# 2) Limpieza mínima: normalizar tipos numéricos y fechas
#    (ajusta columnas si ya conoces sus nombres)
for col in df.columns:
    # intenta número
    c_num = pd.to_numeric(df[col], errors="ignore")
    if not isinstance(c_num, pd.Series) or c_num.dtype != "O":
        df[col] = c_num
        continue
    # intenta fecha (si parece fecha)
    if df[col].dtype == "O" and df[col].str.match(r"\d{4}-\d{2}-\d{2}").fillna(False).mean() > 0.2:
        df[col] = pd.to_datetime(df[col], errors="coerce")

# 3) Muestreo opcional si hay millones de filas
if SAMPLE is not None and len(df) > SAMPLE:
    df = df.sample(SAMPLE, random_state=42).reset_index(drop=True)

# 4) Crear perfil (minimal=True si es muy pesado)
profile = ProfileReport(
    df,
    title="Perfil de Datos – Minería",
    explorative=True,    # interfaz rica (tabs)
    minimal=False,       # aqui lo podriamos poner en True si el dataset es muy grande
    correlations={
        "pearson": {"calculate": True},
        "spearman": {"calculate": True},
        "phi_k": {"calculate": False},  # activa si hay muchas categóricas
        "cramers": {"calculate": True}
    },
    interactions={"continuous": True},
    duplicates={"head": 10},
    samples={"head": 10, "tail": 10}
)

profile.to_file(REPORT)
print(f"Reporte generado: {REPORT}")
print(f"Forma: {df.shape[0]} filas x {df.shape[1]} columnas")


C:\Users\patri\AppData\Local\Temp\ipykernel_20952\2615385722.py:33: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  c_num = pd.to_numeric(df[col], errors="ignore")
C:\Users\patri\AppData\Local\Temp\ipykernel_20952\2615385722.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  if df[col].dtype == "O" and df[col].str.match(r"\d{4}-\d{2}-\d{2}").fillna(False).mean() > 0.2:


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:00<00:00, 224.06it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Reporte generado: profile.html
Forma: 13174 filas x 50 columnas
